In [1]:
import math, random, sys, copy
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import hinge_loss
from sklearn import metrics
import datetime
from datetime import date

In [3]:
def convert_label(x):
#     if x >='7':
#         return 1
#     else:
#         return 0
    return x

In [4]:
def euclidean_distance(x, y):
#     finale_distance = 0
#     for index in range(len(x)):
#         finale_distance = finale_distance + math.pow((x[index] - y[index]), 2)
#     return math.sqrt(finale_distance)
    return np.sqrt(np.sum(np.power(x - y, 2), axis=0))

In [9]:
def pre_process(input_read):
    #DESCRIPTION:
    #Preprocess dataset: Can be done differently
    i = 0
    dataset = np.empty((len(input_read),4),dtype=float)
    for x in input_read:
        x = x[:-1] #Removing the last \n 
        x = x.split(",")
#         if x[-1]=='"male"':
#             x[-1]=1
#         else:
#             x[-1]=0
        dataset[i] = np.asarray([x])
        i += 1
    return np.unique(dataset,axis=0)

In [10]:
# def protras(input_set, epsilon):
# #     from pdb import set_trace; set_trace()
#     #PARAMETERS
#     T = input_set #input
#     T_sign = [True]* len(input_set)
#     init_idx = random.randrange(0,len(input_set) - 1) #doing the random
#     T_rep = dict() #T(yk)
#     T_rep[init_idx] = {init_idx} #save as index
#     S = {init_idx} #output = coreset (save as index)
#     T_sign[init_idx] = False
#     while True: #Repeat
#         #STEP 1: FINDING CLOSET REPRESENTATIVE POINT FOR EACH POINTS OF 
#         #t THAT DOES NOT BELONG TO s
#         for x in range(len(T)): #x removed from T (due to S) at line 79
#             if T_sign[x] != False:
#                 d_min = sys.float_info.max
#                 y_min_idx = 0
#                 # needed improvement
#                 for i in S:
#                     distance = euclidean_distance(T[x][:-1],T[i][:-1]) #improvement
#                     if distance < d_min:
#                         d_min = distance
#                         y_min_idx = i
#                 T_rep[y_min_idx].add(x)
#         #STEP 2: FOR EACH REPRESENTATIVE IN S, FIND THE MAXIMUM DISTANCE
#         #AND ITS CORRESPONDING POINT IN [T_rep]
#         max_wd = 0; cost = 0; y_opt = 0; x_opt = 0
#         d_max = dict(); x_max = dict()
#         for y in S:
#             d_max_tmp = 0
#             x_idx_tmp = 0
#             for x in T_rep[y]:
#                 distance = euclidean_distance(T[x][:-1],T[y][:-1])
#                 if distance > d_max_tmp:
#                     d_max_tmp = distance
#                     x_idx_tmp = x
#             x_max[y] = x_idx_tmp
#             d_max[y] = d_max_tmp
#             pk = len(T_rep[y])*d_max[y]
#             if pk > max_wd:
#                 max_wd = pk
#                 y_opt = y
#             cost = cost + pk
#         cost = cost / len(input_set)
#         print(cost)

#         #CHOOSE AMONG THE POINT ONE THAT IS THE FARTHEST FROM IT REPRESENTATIVE
#         #AND SAVE IT IN CORESET [S]
#         x_opt = x_max[y_opt]
#         S.add(x_opt)
#         T_sign[x_opt] = False
#         T_rep[x_opt] = {x_opt}
#         for i in T_rep.keys():
#             if x_opt in T_rep[i] and x_opt != i:
#                 T_rep[i].remove(x_opt)
#         #CHECKING COST CONDITION
#         if cost < epsilon:
#             coreset = [T[i] for i in S]
#             return coreset,cost

In [13]:
input_read = None
with open("./skin/data_train.csv", 'r') as input_file:
    input_read = input_file.readlines()

In [14]:
dataset = pre_process(input_read)

In [15]:
# coreset, cost = protras(dataset, 0.02)

In [16]:
len(dataset)

44892

In [19]:
int(44892*30/100)

13467

In [21]:
data_size = []
x=448
while x <= 13467:
    data_size.append(x)
    x += 448
number_file = 0

In [22]:
# list_epsilon = [2,1.5,1,0.8,0.5,0.3,0.1,0.09,0.05,0.03]
# number_file = 0

In [23]:
def check_cost(size):
    for x in data_size:
        if size >= x:
            return True
    return False

In [24]:
def CoresetConstruction(X, epsilon):
    global data_size
    global number_file
    n = X.__len__()     # number of data's elements
    C = []              # coresets
    C_count = -1        # number of element in C
    # X_info: contain all info of each element in X
    # [] = index in X; [0] = closest center; [1] = distance to closest center
    # if [0] = -1 >> this point is a center
    X_info = []
    for i in range(n):
        X_info.insert(i, [-1, 1000000.0])
    X_info = np.asarray(X_info)

    #Initialization
    pos_min = np.argmin(X)
    new_center = int(pos_min / X.shape[1])
    C += [new_center]
    X_info[new_center][1] = -1
    C_count += 1
    cost = 100

    while cost > epsilon:
        # Find nearest center of each point in data
        new_center = C[C_count]
        for i in range(n):
            if i not in C:
                new_dist = np.sqrt(np.sum(np.power(X[i][:-1] - X[new_center][:-1], 2), axis=0))
                if new_dist < X_info[i][1]:
                    X_info[i][0] = C_count
                    X_info[i][1] = new_dist

        # Find farthest point in each cluster
        MaxWD = -1          # Measurement to omit noise
        farthest = -1       # index of farthest point
        cost = 0
        for i in range(C_count + 1):
            max_dist = -1
            max_id = -1
            count = 0
            for j in range(n):
                if X_info[j][0]==i:
                    count += 1
                    if max_dist < X_info[j][1]:
                        max_dist = X_info[j][1]
                        max_id = j
            temp = max_dist * count
            if temp > MaxWD:
                MaxWD = temp
                farthest = max_id
            cost += temp / n
        C += [farthest]
        C_count += 1
        X_info[farthest][0] = -1
        X_info[farthest][1] = -1
        if check_cost(len(C)) is True:
            number_file +=1
            path='./skin/'
            filename = path+'coreset'+ str(number_file)+'.csv'
            np.savetxt(filename,dataset[np.asarray(C)],fmt="%f",delimiter=',')
            data_size=data_size[1:]
            print(cost,len(C))
    return np.asarray(C), np.asarray(X_info)

In [ ]:
coreset, cost= CoresetConstruction(dataset,0.000000000003)

In [89]:
# list_epsilon = [2,1.5,1,0.8,0.5,0.3,0.1,0.09,0.05,0.03]
# i = 0
# path='./creditCard/'
# for epsilon in list_epsilon:
#     i += 1
#     coreset, cost= CoresetConstruction(dataset,epsilon)
#     filename = path+'coreset'+ str(i)+'.csv'
#     np.savetxt(filename,dataset[coreset],fmt="%f",delimiter=',')
#     print(len(coreset))